In [41]:
import numpy as np
import pandas as pd
import re

from email.utils import parseaddr

df = pd.read_csv("../dataset/raw/CEAS_08.csv")


In [ ]:
def extract_features(row):
  features = {}

  # Sender Format: Full Name <Local@Domain>
  sender = str(row["sender"])
  sender_name, sender_email = parseaddr(sender)
  sender_domain = sender_email.split("@")[-1].lower() if "@" in sender_email else ""
  
  features["sender_domain"] = sender_domain
  features["sender_has_numbers"] = int(bool(re.search(r"\d", sender)))
  features["sender_domain_length"] = len(sender_domain)

  # Receiver Format: Local@Domain
  receiver = str(row["receiver"])
  receiver_local, receiver_domain = receiver.split("@").lower() if "@" in receiver else ""
  features["receiver_domain"] = receiver_domain
  features["domain_mismatch"] = int(sender_domain != receiver_domain)

  # Subject
  subject = str(row["subject"]).lower()

  features["subject_length"] = len(subject)
  features["subject_num_exclamations"] = subject.count("!")
  features["subject_num_caps"] = sum(1 for c in row["subject"] if c.isupper())
  features["subject_has_spam_words"] = int(any(
      w in subject for w in ["free", "winner", "money", "urgent", "offer", "click"]
  ))

  # Body
  body = str(row["body"]).lower()
  features["body_length"] = len(body)
  features["body_num_links"] = len(re.findall(r"https?://", body))
  features["body_num_exclamations"] = body.count("!")
  features["body_has_spam_words"] = int(any(
      w in body for w in ["free", "credit", "win", "click", "verify", "password"]
  ))

  # Date
  date_str = str(row["date"])
  try:
      dt = pd.to_datetime(date_str, errors="coerce")
      features["email_hour"] = dt.hour
      features["email_day_of_week"] = dt.dayofweek
  except:
      features["email_hour"] = -1
      features["email_day_of_week"] = -1

  return features

In [43]:
features_df = df.apply(extract_features, axis=1)
features_df = pd.DataFrame(features_df.tolist())
features_df.head()

ValueError: not enough values to unpack (expected 2, got 1)